<a href="https://colab.research.google.com/github/pmonda/rpi5-yolo-nn/blob/colab/src/autodidactic_neurosurgeon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os
sys.path.append('/content/drive/MyDrive/ML_Research')
from tiny_yolo import *


if __name__ == "__main__":
    from PIL import Image
    from utils import *

    model = TinyYoloNet()
    model.float()
    model.load_weights('/content/drive/MyDrive/ML_Research/yolov2-tiny-voc.weights')
    model.eval()
    # print(m)

    use_cuda = 1
    if use_cuda:
        model.cuda()

    img = Image.open('/content/drive/MyDrive/ML_Research/pmh2mmc4.png').convert('RGB')
    sized = img.resize((416, 416))

    boxes = do_detect(model, sized, 0.5, 0.5, use_cuda=True)

    class_names = load_class_names('/content/drive/MyDrive/ML_Research/voc.names')
    plot_boxes(img, boxes, 'predict1.jpg', class_names)

/content/drive/MyDrive/ML_Research/utils.py:340: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes()))
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


dog: 1.000000
save plot results to predict1.jpg
